In [1]:
import telnetlib
import json

host = "socket.cryptohack.org"
port = int(13386)

def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline()
    return json.loads(line.decode())

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)

def get_data():
    opening = readline()
    json_send({
        "option" : "get_flag"
    })
    data = json_recv()
    return data

In [2]:
def trimZero(f):
    while len(f) > 0 and f[-1] == 0:
        f.pop()
    return f

def fillZero(f, lenFill):
    assert len(f) <= lenFill
    return f + [0] * (lenFill - len(f))
        
def modPoly(f, modulus):
    r = f.copy()
    for i in range(len(f)):
        r[i] %= modulus
        r[i] = int(r[i])
    return trimZero(r)

def makeMonic(f, modulus):
    r = f.copy()
    leadCoefficientInverse = inverse_mod(r[-1], modulus)
    for i in range(len(r)):
        r[i] *= leadCoefficientInverse
        r[i] %= modulus
        r[i] = int(r[i])
    return r

def addPoly(f1, f2):
    f3 = [0] * max(len(f1), len(f2))
    for i in range(len(f3)):
        if i < min(len(f1), len(f2)):
            f3[i] = f1[i] + f2[i]
        elif len(f1) < len(f2):
            f3[i] = f2[i]
        else:
            f3[i] = f1[i]
    return trimZero(f3)

def mulPoly(f1, f2):
    f3 = [0] * (len(f1) + len(f2))
    for i2 in range(len(f2)):
        for i1 in range(len(f1)):
            f3[i1 + i2] += f1[i1] * f2[i2]
    return trimZero(f3)

def powPoly(f, p):
    r = [1]
    for i in range(p):
        r = mulPoly(r, f)
    return r

def getRoots(f):
    _x_ = PolynomialRing(RationalField(), 'x').gen()
    fInPolyRep = 0
    for i in range(len(f)):
        fInPolyRep += f[i] * _x_ ^ i
    try:
#         print("[i] Finding roots of {}...".format(fInPolyRep))
        return fInPolyRep.roots()
    except:
        return []
    
def genPoly(f):
    _x_ = PolynomialRing(RationalField(), 'x').gen()
    P = 0
    for i in range(len(f)):
        P += f[i] * _x_ ^ i
    return P

def coppersmith(f, modulus, m, X=None):    
    # Assure monic
    assert f[-1] == 1
    
    # Get degree & set X if necessary
    degree = len(f) - 1
    assert degree >= 1
    if not X:
        X = int(modulus ^ (1 / degree))
        if degree == 1:
            X -= 1
#         print('[No upperbound is set so the algorithm set it to {}]'.format(X))
    
    print("[] Solving {} = 0 mod {} \n\tfor x < {} \n\tfor m = {}".format(genPoly(f), modulus, X, m))
    
    # Get dim
    dimension = (degree) * (m + 1)

    # Constructs a matrix.
    M = []
    for v in range( 0, m+1 ):
        for u in range( 0, degree ):
            # Generate g(x) function from f(x)
            g = mulPoly( mulPoly( [modulus^(m-v)], [0]*u+[1] ), powPoly(f, v) )
            
            # Create g(xX) from g(x)
            for i in range(len(g)):
                g[i] *= X^i
            
            # Append to row
            M.append( fillZero(g, dimension) )
    print("[i] Generated a matrix size {}x{}".format(len(M), len(M[0])))
    
    # Convert to matrix
    M = matrix(M)
#     print("[] M:\n")
#     print(M)
    
    # Apply LLL
    M = M.LLL()
#     print("\n[] M.LLL():\n")
#     print(M); print('\n')
    
    
    # Scroll through the shortest vectors as the new polynomials
    roots = []
    for row in M:
        f = trimZero([int(col) for col in row])
        for i in range(len(f)):
            assert f[i] % (X^i) == 0
            f[i] //= X^i
#         print("[] f: {}, row: {}".format(f, row))
        roots += getRoots(f)
    
    # Return unique X component of roots
    return list(dict.fromkeys([root[0] for root in roots]))

def getValue(f, x):
    value = 0
    for coefficient in f[::-1]:
        value += coefficient
        value *= x
    return value // x

def hastad(fs, Ns, m, X=None):
    for i in range(len(Ns)):
        for j in range(i+1, len(Ns)):
            if gcd(Ns[i], Ns[j]) != 1:
                raise ValueError("The Ns must be all coprime!")
    
    # Setup variables
    Ts    = [1] * len(fs)
    ProdN = 1
    for i in range(len(fs)):
        ProdN *= Ns[i]
        
    # Generate Ti, Chinese Remainder Coefficients
    for i in range(len(fs)):
        Ts[i] *= ProdN // Ns[i]
        Ts[i] *= int(inverse_mod(int(Ts[i] % Ns[i]), Ns[i]))
        
    # Make sure T serves our criteria
    for i in range(len(Ts)):
        for j in range(len(Ns)):
            if (i == j and Ts[i] % Ns[j] != 1) or (i != j and Ts[i] % Ns[j] != 0):
                assert False
        
    # Generate f(x) from fi(x)s
    f = [0]
    for i in range(len(fs)):
        f = addPoly(f, mulPoly([Ts[i]], fs[i]))
    f = modPoly(f, ProdN)
    
    # Make sure we actually had monic polynomial
    assert f[-1] == 1
    
    # Using coppersmith to solve
    return coppersmith(f, ProdN, m, X)

In [3]:
fs = []
Ns = []
e  = 11
for _ in range(e+1):
    tn = telnetlib.Telnet(host, port)
    data = get_data()

    c    = int(data["encrypted_flag"])                        # (a*m+b)^11
    N    = int(data["modulus"])                               # N
    a, b = int(data["padding"][0]), int(data["padding"][1])   # a, b

    f = modPoly(powPoly([b, a], e), N)
    f[0] -= c
    f = modPoly(f, N)
    
    fs.append(makeMonic(f, N))
    Ns.append(N)

In [4]:
def hastad2(fs, Ns, eps=1/8):
    for i in range(len(Ns)):
        for j in range(i+1, len(Ns)):
            if gcd(Ns[i], Ns[j]) != 1:
                raise ValueError("The Ns must be all coprime!")
    
    # Setup variables
    Ts    = [1] * len(fs)
    ProdN = 1
    for i in range(len(fs)):
        ProdN *= Ns[i]
        
    # Generate Ti, Chinese Remainder Coefficients
    for i in range(len(fs)):
        Ts[i] *= ProdN // Ns[i]
        Ts[i] *= int(inverse_mod(int(Ts[i] % Ns[i]), Ns[i]))
        
    # Make sure T serves our criteria
    for i in range(len(Ts)):
        for j in range(len(Ns)):
            if (i == j and Ts[i] % Ns[j] != 1) or (i != j and Ts[i] % Ns[j] != 0):
                assert False
        
    # Generate f(x) from fi(x)s
    f = [0]
    for i in range(len(fs)):
        f = addPoly(f, mulPoly([Ts[i]], fs[i]))
    f = modPoly(f, ProdN)
    
    # Make sure we actually had monic polynomial
    assert f[-1] == 1
    
    # Using coppersmith to solve
    P = genPoly(f).change_ring(Zmod(ProdN))
    return P.small_roots(eps=eps)

In [ ]:
hastad(fs, Ns, 4, min(Ns))

[] Solving x^11 + 1580973488224950006117930104295539233623631201590654551420089990938635678109403728632353533431487857875875868043832310979817013594783179496806013310796832760534280978976524125592980260070700100378043257943913705914070407459406752584037323952453062776147103944928179438140918028587196456644367945879051173058436785727551920606064868209459484050364262572003527608104900353593756118163857709147241457160627790070940184888950482998548965812939535839307314390549590791766925125931987163328474127924598485837476351726314270292892374536303707644341183424736689297114862876205341775175773591607194584083587272044242857709475993017434871637584820380120600819361075774586538387929000184397762489898867544351275871758778376179333824824214763004645120464013068881243408823495603802582455269941083150308706470811526094641958495213774332554142942643681001082015247971847437539724406670685351100755961141933366762784113225171878553145916892984986219959032517702573062856405323220821444943934547918